In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
RAW_CSV     = "../data/raw/MobiAct_combined.csv"
OUT_FOLDER  = "raw_windows_test"
WINDOW_SIZE = 800
STEP_SIZE   = 400

In [3]:
dtype_map = {
    "subject_id": "int16", "trial": "int16",
    "acc_x": "float32","acc_y": "float32","acc_z": "float32",
    "gyro_x": "float32","gyro_y": "float32","gyro_z": "float32",
    "azimuth": "float32","pitch": "float32","roll": "float32",
    "label": "category"
}
sensor_cols = [
    "acc_x","acc_y","acc_z",
    "gyro_x","gyro_y","gyro_z",
    "azimuth","pitch","roll"
]

In [4]:
# LABEL MAPPING FOR FILENAME ONLY
def map_labels(raw_label):
    if raw_label in {"BSC","FKL","SDL","FOL"}:
        return "FALL"
    elif raw_label == "LYI":
        return "POST_FALL"
    else:
        return "ADL"

In [5]:
# ——— WINDOW CREATION ———
def create_windows(df, window_size, step_size):
    Xw, yw = [], []
    data = df[sensor_cols].values
    labels = df["fall_label"].values
    n = len(df)
    for start in range(0, n - window_size + 1, step_size):
        block = data[start : start + window_size]
        block_labels = labels[start : start + window_size]
        unique = set(block_labels)
        if "FALL" in unique:
            label = "FALL"
        elif "ADL" in unique:
            label = "ADL"
        else:
            continue
        Xw.append(block)
        yw.append(label)
    return np.array(Xw), np.array(yw)

In [6]:
# ——— LOAD RAW DATA & SPLIT TEST SUBJECTS ———
df = pd.read_csv(RAW_CSV, dtype=dtype_map)
groups = df["subject_id"].values

In [7]:
# split off 20% subjects for test
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
_, test_idx = next(gss.split(df, groups=groups))
df_test = df.iloc[test_idx].reset_index(drop=True)

In [8]:
# map raw labels for window‐level assignment
df_test["fall_label"] = df_test["label"].apply(map_labels)

In [9]:
# ——— CREATE WINDOWS ———
X_windows, y_windows = create_windows(df_test, WINDOW_SIZE, STEP_SIZE)


In [10]:
import os

os.makedirs(OUT_FOLDER, exist_ok=True)

for i, (window, label) in enumerate(zip(X_windows, y_windows)):
    out_df = pd.DataFrame(window, columns=sensor_cols)
    fname  = f"window_800_{i:03d}_{label}.csv"
    out_df.to_csv(os.path.join(OUT_FOLDER, fname), index=False)

print(f"Exported {len(X_windows)} raw windows into '{OUT_FOLDER}/'")


Exported 8925 raw windows into 'raw_windows_test/'
